# Ingest circuits.csv file

In [0]:
dbutils.widgets.help()

dbutils.widgets provides utilities for working with notebook widgets. You can create
different types of widgets and get their bound value.

For more info about a method, use dbutils.widgets.help("methodName") .
 combobox(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a combobox input widget with a given name, default value and choices dropdown(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a dropdown input widget a with given name, default value and choices get(name: String): String -> Retrieves current value of an input widget getArgument(name: String, optional: String): String -> (DEPRECATED) Equivalent to get multiselect(name: String, defaultValue: String, choices: Seq, label: String): void -> Creates a multiselect input widget with a given name, default value and choices remove(name: String): void -> Removes an input widget from the notebook removeAll: void -> Removes all widgets in the notebook text(name: String, defaultValue: String, label: String): void -> Creates a text input widget with a given name and default value

In [0]:
dbutils.widgets.dropdown("Environment", "Dev", ["Prod", "Dev", "Test"], "Environment")
env = dbutils.widgets.get("Environment")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Step 1 - Read the CSV file using spark dataframe reader

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [0]:
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True),
                                    
                                    ])

In [0]:
circuits_df = spark.read.csv(f"{raw_folder_path}/{v_file_date}/circuits.csv", header=True, schema=circuits_schema)

In [0]:
# display(circuits_df)
circuits_df.show(5)

+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
|circuitId| circuitRef|                name|    location|  country|     lat|    lng|alt|                 url|
+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
|        1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|144.968| 10|http://en.wikiped...|
|        2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|101.738| 18|http://en.wikiped...|
|        3|    bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|50.5106|  7|http://en.wikiped...|
|        4|  catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|2.26111|109|http://en.wikiped...|
|        5|   istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517| 29.405|130|http://en.wikiped...|
+---------+-----------+--------------------+------------+---------+--------+-------+---+--------------------+
only showi

### Step 2 - Select only required columns

In [0]:
from pyspark.sql.functions import col, lit

In [0]:
circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country"), col("lat"), col("lng"), col("alt"))

In [0]:
# display(circuits_selected_df)
circuits_selected_df.show(5)

+---------+-----------+--------------------+------------+---------+--------+-------+---+
|circuitId| circuitRef|                name|    location|  country|     lat|    lng|alt|
+---------+-----------+--------------------+------------+---------+--------+-------+---+
|        1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|144.968| 10|
|        2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|101.738| 18|
|        3|    bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|50.5106|  7|
|        4|  catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|2.26111|109|
|        5|   istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517| 29.405|130|
+---------+-----------+--------------------+------------+---------+--------+-------+---+
only showing top 5 rows



### Step 3 - Rename the columns as required

In [0]:
circuits_renamed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id").withColumnRenamed("circuitRef", "circuit_ref").withColumnRenamed("lat", "latitude").withColumnRenamed("lng", "longitude").withColumnRenamed("alt", "altitude").withColumn("env", lit(env)).withColumn("file_date", lit(v_file_date))

In [0]:
# display(circuits_renamed_df)
circuits_renamed_df.show(5)

+----------+-----------+--------------------+------------+---------+--------+---------+--------+----+----------+
|circuit_id|circuit_ref|                name|    location|  country|latitude|longitude|altitude| env| file_date|
+----------+-----------+--------------------+------------+---------+--------+---------+--------+----+----------+
|         1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968|      10|Prod|2021-03-21|
|         2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738|      18|Prod|2021-03-21|
|         3|    bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|       7|Prod|2021-03-21|
|         4|  catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|     109|Prod|2021-03-21|
|         5|   istanbul|       Istanbul Park|    Istanbul|   Turkey| 40.9517|   29.405|     130|Prod|2021-03-21|
+----------+-----------+--------------------+------------+---------+--------+---------+--------+

### Step 4 - Adding ingestion date to the dataframe

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
# circuits_final_df = circuits_renamed_df.withColumn("ingestion_date", current_timestamp())
circuits_final_df = add_ingestion_date(circuits_renamed_df)

In [0]:
# display(circuits_final_df)
circuits_final_df.show(5)

+----------+-----------+--------------------+------------+---------+--------+---------+--------+----+----------+--------------------+
|circuit_id|circuit_ref|                name|    location|  country|latitude|longitude|altitude| env| file_date|      ingestion_date|
+----------+-----------+--------------------+------------+---------+--------+---------+--------+----+----------+--------------------+
|         1|albert_park|Albert Park Grand...|   Melbourne|Australia|-37.8497|  144.968|      10|Prod|2021-03-21|2023-06-03 22:29:...|
|         2|     sepang|Sepang Internatio...|Kuala Lumpur| Malaysia| 2.76083|  101.738|      18|Prod|2021-03-21|2023-06-03 22:29:...|
|         3|    bahrain|Bahrain Internati...|      Sakhir|  Bahrain| 26.0325|  50.5106|       7|Prod|2021-03-21|2023-06-03 22:29:...|
|         4|  catalunya|Circuit de Barcel...|    Montmeló|    Spain|   41.57|  2.26111|     109|Prod|2021-03-21|2023-06-03 22:29:...|
|         5|   istanbul|       Istanbul Park|    Istanbul|   T

### Step 5 - Write data to datalake as Parquet

In [0]:
# circuits_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/circuits.parquet")
circuits_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.circuits")

In [0]:
df = spark.read.parquet(f"{processed_folder_path}/circuits")

In [0]:
display(df)
# df.show()

circuit_id,circuit_ref,name,location,country,latitude,longitude,altitude,ingestion_date
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,2023-06-02T01:18:02.656+0000
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,2023-06-02T01:18:02.656+0000
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,2023-06-02T01:18:02.656+0000
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,2023-06-02T01:18:02.656+0000
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,2023-06-02T01:18:02.656+0000
6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,2023-06-02T01:18:02.656+0000
7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,13,2023-06-02T01:18:02.656+0000
8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,228,2023-06-02T01:18:02.656+0000
9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,153,2023-06-02T01:18:02.656+0000
10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.3278,8.56583,103,2023-06-02T01:18:02.656+0000


In [0]:
%sql
SELECT * FROM f1_processed.circuits;

circuit_id,circuit_ref,name,location,country,latitude,longitude,altitude,env,file_date,ingestion_date
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968,10,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.738,18,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.0325,50.5106,7,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57,2.26111,109,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
5,istanbul,Istanbul Park,Istanbul,Turkey,40.9517,29.405,130,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
6,monaco,Circuit de Monaco,Monte-Carlo,Monaco,43.7347,7.42056,7,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
7,villeneuve,Circuit Gilles Villeneuve,Montreal,Canada,45.5,-73.5228,13,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
8,magny_cours,Circuit de Nevers Magny-Cours,Magny Cours,France,46.8642,3.16361,228,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
9,silverstone,Silverstone Circuit,Silverstone,UK,52.0786,-1.01694,153,Prod,2021-03-21,2023-06-03T22:29:17.510+0000
10,hockenheimring,Hockenheimring,Hockenheim,Germany,49.3278,8.56583,103,Prod,2021-03-21,2023-06-03T22:29:17.510+0000


In [0]:
dbutils.notebook.exit("Success")

Success